In [101]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = "C:/Users/Adrian.Wiegman/Documents/GitHub/Wiegman_USDA_ARS/MEP/_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic('pinfo2', '$name')
File:      c:\workspace\geodata\mep\<ipy

# Watershed delineation

This notebook determines the catchment size for each cranberry bog within an MEP subwatershed. 

1. Generate a point at the center of the low elevation zone of each cranberry bog.
2. Delineate a topographic watershed for each cranberry bog. 
3. Intersect the topographic watershed with the groundwater drainage for each bog.
4. Add the area to the of each to the pour point 

## 1. Generate a point at the center of the lowest elevation zone of each cranberry bog.

This is done by generating a centroid point inside the bog polygon after the polygon has been clipped to contain only the lowest 5th percentile elevation. 

In [3]:
# extract lidar by mask of cranberry polygons
out_raster = arcpy.sa.ExtractByMask(
    in_raster="Extract_LiDA1",
    in_mask_data="WMAbogsDRAFT2013",
    extraction_area="INSIDE",
    analysis_extent='-7924127.03810333 5051155.01774517 -7786374.58693344 5173790.85434187 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'
)
out_raster.save(r"C:\Workspace\Geodata\MEP\Default.gdb\Cran_Lidar")

In [ ]:
# raster to point 
arcpy.conversion.RasterToPoint(
    in_raster=r"C:\Workspace\Geodata\MEP\Default.gdb\Cran_Lidar",
    out_point_features=r"C:\Workspace\Geodata\MEP\Default.gdb\Cran_Lid_Points",
    raster_field="Value"
)

In [ ]:
# identity 
arcpy.analysis.Identity(
    in_features="Cran_Lid_Points",
    identity_features="WMAbogsDRAFT2013",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\Cran_Lid_Points_Identity",
    join_attributes="ONLY_FID",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

In [6]:
# Summarize bottom 5th percentile elevation inside bogs. 
raster = "Cran_Lidar"
poly = "WMAbogsDRAFT2013"
zonefield = "FID"
pct = 5  # 5% percentile
outname = "Lid_Bog_ZS"
Lid_bog_ZS = arcpy.ia.ZonalStatistics(poly, 
                                      zonefield, 
                                      raster, 
                                      "PERCENTILE", 
                                      "DATA", 
                                      "CURRENT_SLICE", 
                                      pct, 
                                      "AUTO_DETECT"); 
Lid_bog_ZS.save(os.path.join(outname))

In [7]:
a = "Cran_Lidar"
b = "Lid_Bog_ZS"
print(a,b)
lidar_bog_le5pct = arcpy.ia.LessThanEqual(a,b); 
lidar_bog_le5pct.save("lid_bog_le5pct")

Cran_Lidar Lid_Bog_ZS


In [8]:
# convert raster of lidar_le5pct to polygon
outfile = "ele5pct_bog_poly"
poly = arcpy.conversion.RasterToPolygon("lidar_le5pct", outfile, "SIMPLIFY", "VALUE", "SINGLE_OUTER_PART", None)

In [ ]:
# select only low elevation zones
selection = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="lidar_bog_le5pct_poly",
    selection_type="NEW_SELECTION",
    where_clause="gridcode = 1",
    invert_where_clause=None
)
# make a copy of the selection
copy = arcpy.management.CopyFeatures(
    in_features="lidar_bog_le5pct_poly",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5pct_poly_Copy",
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)

In [11]:
# use identity intersection to add bog FIDs to the polygon
arcpy.analysis.Identity(
    in_features="lidar_bog_le5pct_poly_Copy",
    identity_features="WMAbogsDRAFT2013",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5pct_po_Identity",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\lidar_bog_le5pct_po_Identity'>

In [ ]:
# dissolve the layer by cranberry bog FID
arcpy.management.Dissolve(
    in_features="lidar_bog_le5pct_po_Identity",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5pct_po_Dissolve",
    dissolve_field="FID_WMAbogsDRAFT2013",
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

In [13]:
# Feature to point to make centroid within low elevation zone of each bog
arcpy.management.FeatureToPoint(
    in_features="lidar_bog_le5pct_po_Dissolve",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5_centroids",
    point_location="INSIDE"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\lidar_bog_le5_centroids'>

## 2. Delineate a topographic watershed for each cranberry bog centroid

This is done using two approaches, one witha 10m resolution lidar. 


In [14]:
arcpy.analysis.Identity(
    in_features="lidar_bog_le5_centroids",
    identity_features="subs_le5_cran_Dissolve",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5_centr_Identity",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\lidar_bog_le5_centr_Identity'>

In [ ]:
# this takes quite a long time to run
# ready to use tool uses USGS data 
#https://pro.arcgis.com/en/pro-app/latest/tool-reference/ready-to-use/watershed.htm#:~:text=Usage%201%20Tools%20in%20the%20Ready%20To%20Use,extent%20of%20the%20specified%20data%20source.%20More%20items
arcpy.agolservices.Watershed(
    InputPoints="lidar_bog_le5_centr_Identity",
    PointIDField="FID_WMAbogsDRAFT2013",
    SnapDistance="",
    SnapDistanceUnits="Meters",
    DataSourceResolution="FINEST",
    Generalize=False,
    ReturnSnappedPoints=True
)

In [16]:
# save watershed outputs
arcpy.management.CopyFeatures(
    in_features="Output Snapped Points",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5_centr_Ident_pour",
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)

arcpy.management.CopyFeatures(
    in_features="Output Watershed",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5_centr_watersheds",
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\lidar_bog_le5_centr_watersheds'>

## 4. Intersect the topographic watershed with the groundwater drainage for each bog.

Here I take a layer of cranberry topographic drainages, and clip them by groundwater subwatersheds from MEP reports. This is done to estimate the area draining into each cranberry bog. 

In [136]:
_ = r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5_centr_Identity"
_ = arcpy.da.FeatureClassToNumPyArray(_,['FID_WMAbogsDRAFT2013','FID_MEP_SUBW_NAME'])
pdf1 = pd.DataFrame(_)
print(pdf1)

     FID_WMAbogsDRAFT2013  FID_MEP_SUBW_NAME
0                     546                474
1                     547                474
2                     977                 53
3                     584                332
4                     167                 18
..                    ...                ...
318                   465                117
319                   296                117
320                   700                117
321                   980                117
322                   702                117

[323 rows x 2 columns]


In [103]:
# Obtain the first feature layer from the item
_ = r"C:\Workspace\Geodata\MEP\Default.gdb\lidar_bog_le5_centr_watersheds"
_ = arcpy.da.FeatureClassToNumPyArray(_,'PourPtID')
pdf2 = pd.DataFrame(_)

In [134]:
# select the data frame 
df = pdf1
# create list to store output filenames
outfiles = [None]*len(df)

# loop through all rows of the cranberry topographic watersheds contained dataframe 
for i in range(len(df)):
    bogID = pdf1.iloc[i,0]
    subID = pdf1.iloc[i,1]
    
    # create output file to store the clipped data
    outfile = os.path.join(odr,"bog{}_sub{}.shp".format(bogID,subID))
    outfiles[i] = outfile
    print(outfiles[i])
    
    # select the cranberry bog topographic watershed matching the bogID
    _ = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="lidar_bog_le5_centr_watersheds",
        selection_type="NEW_SELECTION",
        where_clause="PourPtID = {}".format(bogID),
        invert_where_clause=None)
    # save a temporary file with bog watershed
    bog = os.path.join(tdr.name,"bog_{}".format(bogID)
    bog = arcpy.management.CopyFeatures(
        in_features=_,
        out_feature_class=bog)
    
    # Select the MEP subwatershed that the cranberry bog centroid resides in
    _ = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="MEP_SUBW_NAME",
        selection_type="NEW_SELECTION",
        where_clause="OBJECTID = {}".format(subID),
        invert_where_clause=None)
    sub = os.path.join(tdr.name,"sub_{}".format(subID))
    sub = arcpy.management.CopyFeatures(
        in_features=_,
        out_feature_class=sub)
    
    # Clip the bog layer by the subwatershed 
    clip = arcpy.gapro.ClipLayer(
        input_layer=bog,
        clip_layer=sub,
        out_feature_class=os.path.join(odr,outfiles[i]))
                       
    # add bog id and sub id to clip attribute table
    arcpy.management.AddField(clip, "subID", "TEXT", "", "", "20")
    arcpy.management.AddField(clip, "bogID", "TEXT", "", "", "20")
    arcpy.management.CalculateField(clip, "subID", 
                                    '{}'.format(subID), "PYTHON3")
    arcpy.management.CalculateField(clip, "bogID", 
                                    '{}'.format(bogID), "PYTHON3")
    # end loop
                                             
print("\n\nclipping complete!\nnow merging clipped watersheds")
                       
# merge all clipped bog watershed files and save to default geodatabase                       
arcpy.management.Merge(
    inputs=outfiles,
    output=r"C:\Workspace\Geodata\MEP\Default.gdb\bog_subs")

C:\Workspace\Geodata\MEP\outputs\bog546_sub474.shp
C:\Workspace\Geodata\MEP\outputs\bog547_sub474.shp
C:\Workspace\Geodata\MEP\outputs\bog977_sub53.shp
C:\Workspace\Geodata\MEP\outputs\bog584_sub332.shp
C:\Workspace\Geodata\MEP\outputs\bog167_sub18.shp
C:\Workspace\Geodata\MEP\outputs\bog162_sub367.shp
C:\Workspace\Geodata\MEP\outputs\bog582_sub332.shp
C:\Workspace\Geodata\MEP\outputs\bog166_sub187.shp
C:\Workspace\Geodata\MEP\outputs\bog583_sub332.shp
C:\Workspace\Geodata\MEP\outputs\bog243_sub81.shp
C:\Workspace\Geodata\MEP\outputs\bog764_sub109.shp
C:\Workspace\Geodata\MEP\outputs\bog165_sub601.shp
C:\Workspace\Geodata\MEP\outputs\bog631_sub601.shp
C:\Workspace\Geodata\MEP\outputs\bog629_sub601.shp
C:\Workspace\Geodata\MEP\outputs\bog194_sub81.shp
C:\Workspace\Geodata\MEP\outputs\bog630_sub601.shp
C:\Workspace\Geodata\MEP\outputs\bog164_sub601.shp
C:\Workspace\Geodata\MEP\outputs\bog193_sub482.shp
C:\Workspace\Geodata\MEP\outputs\bog585_sub684.shp
C:\Workspace\Geodata\MEP\outputs\bo

C:\Workspace\Geodata\MEP\outputs\bog702_sub117.shp


clipping complete!
now merging clipped watersheds


<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\bog_subs'>